#### Import the necessary package

In [13]:
import make_env as world
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import random

#### load the environment

In [14]:
env = world.make_env('simple')

In [15]:
env.discrete_action_input = True
for i in range(100):
    val =  env.render(mode='Test')
    val = env.step(action_n=[np.random.choice([0,1,2,3,4])])

#### Create the model

In [16]:
agent_move = Sequential()

In [17]:
agent_move.add(Dense(64,activation='relu', input_dim = 5))
agent_move.add(Dense(50,activation='relu'))
agent_move.add(Dense(32,activation='relu'))
agent_move.add(Dense(24,activation='relu'))
agent_move.add(Dense(16,activation='relu'))
agent_move.add(Dense(10,activation='relu'))
agent_move.add(Dense(8, activation='relu'))
agent_move.add(Dense(4,activation='relu'))
agent_move.add(Dense(1,activation='linear'))

In [18]:
agent_move.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 64)                384       
_________________________________________________________________
dense_20 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_22 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_23 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_24 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 88        
__________

In [19]:
agent_move.compile(loss='mean_squared_error', optimizer='Adam')

#### Introduce target network

In [20]:
agent_target = Sequential()

In [21]:
agent_target.add(Dense(64,activation='relu', input_dim = 5))
agent_target.add(Dense(50,activation='relu'))
agent_target.add(Dense(32,activation='relu'))
agent_target.add(Dense(24,activation='relu'))
agent_target.add(Dense(16,activation='relu'))
agent_target.add(Dense(10,activation='relu'))
agent_target.add(Dense(8, activation='relu'))
agent_target.add(Dense(4,activation='relu'))
agent_target.add(Dense(1,activation='linear'))

In [22]:
agent_target.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 64)                384       
_________________________________________________________________
dense_29 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_30 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_31 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_32 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_33 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_34 (Dense)             (None, 8)                 88        
__________

In [23]:
agent_move.save_weights('./random_weight')

In [24]:
episode = 1000
max_episode_length = 400
discount_factor = 0.9

In [32]:
for i in range(276,episode):
    
    #replace the experience replay 
    if ((i+1) % 25) == 0:
        episode_experience = episode_experience[-2000:]
        
        #replace old network with new network
        agent_move.save_weights('./weight')
        
        agent_target.load_weights('./weight')
        
        agent_move.load_weights('./random_weight')
    
    if i == 0:
        episode_experience = []
    
    
    epsilon = 1 / (np.sqrt(i+10))
    
    S = env.reset()
    S = S[0]
    
    A = np.random.choice([0,1,2,3,4])

    for j in range(max_episode_length):
        
        observe = env.step(action_n=[A])
        
        # Reward clip so that it values from 1 to -1
        R = observe[1][0] 
        
        if R < -1.5:
            R = -1
        else :
            R = (1.5 + R) / (1.5)
        
        S_new = observe[0][0]
        
        Test = np.array([[S_new[0],S_new[1],S_new[2],S_new[3], 0],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 1],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 2],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 3],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 4]])
        
        #value of all possible action from target network
        all_action_value = agent_move.predict(Test)
        
        #chose action which is maximum value 
        A_new = np.argmax(all_action_value)
        
        
        #epsilon - greedy 
        rand = np.random.uniform([0])
        
        if rand[0] > epsilon:
            A_new = A_new
        else :
            A_new = np.random.choice([0,1,2,3,4])
        
        
        Target = agent_target.predict(Test)[A_new][0]
        
        
        episode_experience.append([S[0],S[1],S[2],S[3],A,R + discount_factor * Target])
            
        S = S_new
        A = A_new
        
        if (j % (200)) == 0 :
            env.reset()
            
            S = env.reset()
            S = S[0]
    
            A = np.random.choice([0,1,2,3,4])
    
        if (i == 0 and j > 65) or (i > 0) :
            minibatch = random.sample(episode_experience,64)
    
            Test = np.array(minibatch)
        
            agent_move.fit(Test[:,0:5], Test[:,5], epochs=3)
    

NameError: name 'episode_experience' is not defined

In [26]:
import time

In [36]:

S = env.reset()
S = S[0]
    
A = np.random.choice([0,1,2,3,4])

epsilon = 0.0
agent_target.load_weights('./weight')
for _ in range(100):
    time.sleep(.5)
    observe = env.step(action_n=[A])
        
    chek =  env.render(mode='Test')
        
    # Reward clip so that it values from 1 to -1
    R = observe[1][0] 
        
    if R < -1.5:
        R = -1
    else :
        R = (1.5 + R) / (1.5)
        
    S_new = observe[0][0]
        
    Test = np.array([[S_new[0],S_new[1],S_new[2],S_new[3], 0],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 1],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 2],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 3],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 4]])
    
   
    #value of all possible action from target network
    all_action_value = agent_target.predict(Test)
        
    #chose action which is maximum value 
    A_new = np.argmax(all_action_value)
    
    rand = np.random.uniform([0])
    if rand[0] > epsilon:
            A_new = A_new
    else :
            A_new = np.random.choice([0,1,2,3,4])
    
    S = S_new
    A = A_new
        

KeyboardInterrupt: 

In [ ]:
env2 = world.make_env('simple_adversary')

In [ ]:
env2.render("test")

In [ ]:
agent2 = env2.agents

In [ ]:
env2.render('Test')
env.step(action_n=[3])

In [ ]:
for i in range(100):
    if (i%10) == 0:
        print(i,'1')
    
    if ((i+1) % 10) == 0:
        print(i, '2')

In [ ]:
env.reset()

In [28]:
i

99

In [ ]:
episode = [[1,2],[2,4],[3,1],[4,2]]

In [ ]:
time.sleep(3)

In [ ]:
random.sample(episode,3)

In [ ]:
i